## **Read the news table from LakeHouse DB**

In [3]:
df = spark.sql("SELECT * FROM bing_lake_db.tbl_latest_news")
display(df)

StatementMeta(, a6d07d2a-06fc-404a-85e0-ba710dc9f711, 5, Finished, Available)

SynapseWidget(Synapse.DataFrame, 2138cfdc-4f6c-4b3b-bfea-b769953bdb07)

## **Import Synapse ML**

In [4]:
import synapse.ml.core
from synapse.ml.services import AnalyzeText

StatementMeta(, a6d07d2a-06fc-404a-85e0-ba710dc9f711, 6, Finished, Available)

## **Configure Synapse ML**

In [5]:
model = (AnalyzeText()
        .setTextCol("description")
        .setKind("SentimentAnalysis")
        .setOutputCol("response")
        .setErrorCol("error"))

StatementMeta(, a6d07d2a-06fc-404a-85e0-ba710dc9f711, 7, Finished, Available)

## **Apply the model**

In [6]:
result = model.transform(df)

StatementMeta(, a6d07d2a-06fc-404a-85e0-ba710dc9f711, 8, Finished, Available)

In [7]:
display(result)

StatementMeta(, a6d07d2a-06fc-404a-85e0-ba710dc9f711, 9, Finished, Available)

SynapseWidget(Synapse.DataFrame, ecbc9634-c589-44cc-b1cb-e0878684721f)

## **Grab sentiment from the response**

In [8]:
#Create Sentiment Column
from pyspark.sql.functions import col

sentiment_df_final = result.withColumn("sentiment", col("response.documents.sentiment"))

StatementMeta(, a6d07d2a-06fc-404a-85e0-ba710dc9f711, 10, Finished, Available)

In [9]:
display(sentiment_df_final)

StatementMeta(, a6d07d2a-06fc-404a-85e0-ba710dc9f711, 11, Finished, Available)

SynapseWidget(Synapse.DataFrame, d3f71ed8-1fa3-4d30-988a-911751ad2ac6)

## **Remove unwanted column**

In [ ]:
sentiment_df_final = sentiment_df.drop("error", "response")

In [ ]:
display(sentiment_df_final)

In [10]:
from pyspark.sql.utils import AnalysisException

try:

    table_name = 'bing_lake_db.tbl_sentiment_analysis'

    sentiment_df_final.write.format("delta").saveAsTable(table_name)

except AnalysisException:

    print("Table Already Exists")

    sentiment_df_final.createOrReplaceTempView("vw_sentiment_df_final")

    spark.sql(f""" MERGE INTO {table_name} target_table
                   USING vw_sentiment_df_final source_view
                   
                   ON source_view.url = target_table.url
                   
                   WHEN MATCHED AND
                   source_view.title <> target_table.title OR
                   source_view.description <> target_table.description OR
                   source_view.category <> target_table.category OR
                   source_view.image <> target_table.image OR
                   source_view.provider <> target_table.provider OR
                   source_view.datePublished <> target_table.datePublished 
                   
                   THEN UPDATE SET *
                   
                   WHEN NOT MATCHED THEN INSERT *
                """)

StatementMeta(, a6d07d2a-06fc-404a-85e0-ba710dc9f711, 12, Finished, Available)

Table Already Exists


In [11]:
display(sentiment_df_final)

StatementMeta(, a6d07d2a-06fc-404a-85e0-ba710dc9f711, 13, Finished, Available)

SynapseWidget(Synapse.DataFrame, be28b231-d1a7-4496-b182-790cdd6e7f56)